In [ ]:
import os

class Item:
    def __init__(self, name, price, stock):
        self._name = name
        self._price = price
        self._stock = stock

    def get_name(self):
        return self._name

    def get_price(self):
        return self._price

    def get_stock(self):
        return self._stock

    def set_stock(self, stock):
        if stock >= 0:
            self._stock = stock

    def __str__(self):
        return f"{self._name}: ${self._price:.2f} (Stock: {self._stock})"


class Product(Item):
    def __init__(self, name, price, stock):
        super().__init__(name, price, stock)


class PaymentProcessor:
    def process_payment(self, amount):
        raise NotImplementedError("Subclasses should implement this method")


class CreditCardPayment(PaymentProcessor):
    def process_payment(self, amount):
        print(f"Processing credit card payment of ${amount:.2f}...")
        return True


class DebitCardPayment(PaymentProcessor):
    def process_payment(self, amount):
        print(f"Processing debit card payment of ${amount:.2f}...")
        return True


class Cart:
    def __init__(self):
        self._items = []

    def add_product(self, product, quantity):
        available_stock = product.get_stock()
        if available_stock > 0:
            if quantity <= available_stock:
                for _ in range(quantity):
                    self._items.append(product)
                product.set_stock(available_stock - quantity)
                print(f"Added {quantity} x {product.get_name()} to the cart. Total price: ${product.get_price() * quantity:.2f}")
            else:
                print(f"Sorry, we only have {available_stock} of {product.get_name()} in stock.")
        else:
            print(f"Sorry, {product.get_name()} is out of stock.")

    def remove_product(self, product_name):
        for item in self._items:
            if item.get_name().lower() == product_name.lower():
                self._items.remove(item)
                item.set_stock(item.get_stock() + 1)  # Increment stock of the product
                print(f"Removed {item.get_name()} from the cart.")
                return
        print(f"{product_name} not found in the cart.")

    def total_price(self):
        return sum(item.get_price() for item in self._items)

    def display_cart(self):
        print("Your Cart:")
        for item in self._items:
            print(item)
        print(f"Total: ${self.total_price():.2f}")

    def save_cart(self, filename):
        with open(filename, 'w') as file:
            for item in self._items:
                file.write(f"{item.get_name()},{item.get_price()},{1}\n")  # Save 1 unit of the item
        print(f"Cart saved to {filename}.")

    def load_cart(self, filename, available_products):
        if os.path.exists(filename):
            with open(filename, 'r') as file:
                lines = file.readlines()
                for line in lines:
                    name, price, quantity = line.strip().split(',')
                    price = float(price)
                    quantity = int(quantity)
                    for product in available_products:
                        if product.get_name().lower() == name.lower():
                            if product.get_stock() >= quantity:
                                for _ in range(quantity):
                                    self._items.append(product)
                                product.set_stock(product.get_stock() - quantity)
                            break
            print(f"Cart loaded from {filename}.")
        else:
            print("No saved cart found.")


class User:
    def __init__(self, username):
        self._username = username
        self._cart = Cart()

    def add_to_cart(self, product, quantity):
        self._cart.add_product(product, quantity)

    def remove_from_cart(self, product_name):
        self._cart.remove_product(product_name)

    def checkout(self):
        total = self._cart.total_price()
        if total > 0:
            print(f"{self._username}, your total is ${total:.2f}.")
            payment_method = input("Enter payment method (credit/debit): ").lower()
            processor = None

            if payment_method == 'credit':
                processor = CreditCardPayment()
            elif payment_method == 'debit':
                processor = DebitCardPayment()

            if processor and self.process_payment(total, processor):
                print("Payment successful! Thank you for your purchase.")
                self._cart._items.clear()
                self._cart.save_cart(f'{self._username}_cart.txt')
            else:
                print("Payment failed. Please try again.")
        else:
            print("Your cart is empty!")

    def process_payment(self, amount, processor: PaymentProcessor):
        return processor.process_payment(amount)


def load_products():
    if os.path.exists("products.txt"):
        products = []
        with open("products.txt", "r") as file:
            for line in file:
                name, price, stock = line.strip().split(',')
                products.append(Product(name, float(price), int(stock)))
        return products
    else:
        print("Products file not found.")
        return []


def save_products(products):
    with open("products.txt", "w") as file:
        for product in products:
            file.write(f"{product.get_name()},{product.get_price()},{product.get_stock()}\n")


def shopping_session():
    products = load_products()
    if not products:
        print("No products available. Exiting.")
        return

    username = input("Enter your username: ")
    user = User(username)

    while True:
        action = input("Would you like to 'add', 'remove', 'checkout', 'load cart', 'save cart','available' or 'exit'? ").lower()

        if action == 'available':
            print("\nAvailable Products:")
            for product in products:
                print(product) 

        if action == 'add':
            product_input = input("Enter the name of the product to add and the quantity (Product name/quantity): ")

            if '/' in product_input:
                product_name, quantity_str = product_input.split('/')
                
                if quantity_str.isdigit():
                    quantity = int(quantity_str)

                    product = next((p for p in products if p.get_name().lower() == product_name.lower()), None)

                    if product:
                        if quantity <= product.get_stock():
                            user.add_to_cart(product, quantity)
                        else:
                            print(f"Sorry, we only have {product.get_stock()} of {product_name} in stock.")
                    else:
                        print(f"Product '{product_name}' not found in the inventory.")
                else:
                    print("Invalid quantity. Please enter a valid integer.")
            else:
                print("Invalid format. Please enter in the form 'Product name/quantity'.")

        elif action == 'remove':
            product_name = input("Enter the name of the product to remove: ")
            user.remove_from_cart(product_name)
        
        elif action == 'checkout':
            user.checkout()
        
        elif action == 'load cart':
            user._cart.load_cart(f'{username}_cart.txt', products)

        elif action == 'save cart':
            user._cart.save_cart(f'{username}_cart.txt')
        
        elif action == 'exit':
            print("Thank you for visiting!")
            save_products(products) 
            break
        
        else:
            print("Invalid action. Please choose again.")



shopping_session()

Enter your username:   Shahzadi
Would you like to 'add', 'remove', 'checkout', 'load cart', 'save cart','available' or 'exit'?  add
Enter the name of the product to add and the quantity (Product name/quantity):  Knife,79.99,90


Invalid format. Please enter in the form 'Product name/quantity'.


Would you like to 'add', 'remove', 'checkout', 'load cart', 'save cart','available' or 'exit'?  add
Enter the name of the product to add and the quantity (Product name/quantity):  Racket,49.99,40


Invalid format. Please enter in the form 'Product name/quantity'.


Would you like to 'add', 'remove', 'checkout', 'load cart', 'save cart','available' or 'exit'?  add
Enter the name of the product to add and the quantity (Product name/quantity):  WaffleMaker,39.99,120


...

Invalid action. Please choose again.


Would you like to 'add', 'remove', 'checkout', 'load cart', 'save cart','available' or 'exit'?  checkout


Your cart is empty!


Would you like to 'add', 'remove', 'checkout', 'load cart', 'save cart','available' or 'exit'?  checkout


Your cart is empty!
